# Multiselect map in panel
This is a panel with strictly python callback. With the date range

In [1]:
import numpy as np
import pandas as pd

import bokeh.io

import panel as pn
import datetime as dt

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

tile_provider = get_provider(CARTODBPOSITRON)

bokeh.io.output_notebook()


Loading BokehJS ...

In [2]:
import inspect

inspect.getfile(pn)

'/opt/miniconda3/envs/basics/lib/python3.9/site-packages/panel/__init__.py'

# Load and prep data

In [3]:
# Helper fxn to set coords right
def mercator_creator(df, lat='lat', lon='lon'):
    # from https://www.youtube.com/watch?v=BojxegBh9_4
    
    k = 6378137
    df['x'] = k * np.radians(df[lon])
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360)) * k
    
    return df

In [4]:
# Load in the metadata
ferm_locations = pd.read_csv("./data/ferm.csv")

# Split the Lat-Long column into two columns, Lat and Long
ferm_locations[['Lat', 'Long']] = ferm_locations['Lat-Long'].str.split(', ', 1, expand=True)

# Create a new dataframe only if the point has a valid Lat and Long
ferm_locations_final = ferm_locations[ferm_locations['Lat'].notna()].copy()

# Convert from a string, and put in columns called Latitude and Longitude
ferm_locations_final.loc[:,'Latitude1'] = pd.to_numeric(ferm_locations_final['Lat'],errors='coerce')
ferm_locations_final.loc[:,'Longitude1'] = pd.to_numeric(ferm_locations_final['Long'],errors='coerce')

# Add x, y for lat long
ferm_locations_final = mercator_creator(ferm_locations_final, lat='Latitude1', lon='Longitude1')

class_options = list(ferm_locations_final.Class.unique())

ferm_locations_final.Acquisition_date = pd.to_datetime(ferm_locations_final.Acquisition_date)
ferm_locations_final.Extract_date = pd.to_datetime(ferm_locations_final.Extract_date)
ferm_locations_final.Arrival_date = pd.to_datetime(ferm_locations_final.Arrival_date)

# Panel

In [5]:
# inital_date_range = dt.datetime(2021, 12, 1), dt.datetime(2022, 8, 1)

In [6]:
def extract_sub_df(df, ferm_class_list, country_list, date_range):
    """Extract sub data frame for country and class over a date range."""
    inds = (
        (df["Class"].isin(ferm_class_list))
        & (df["Country_made"].isin(country_list))
        & (df["Acquisition_date"] >= date_range[0])
        & (df["Acquisition_date"] <= date_range[1])
    )

    return df.loc[inds, :]


In [7]:
# date_range_slider_old = pn.widgets.DateRangeSlider(
#     name='Date Range Slider',
#     start=dt.datetime(2021, 12, 1), end=dt.datetime(2022, 8, 1),
#     value=(dt.datetime(2021, 12, 1), dt.datetime(2022, 8, 1))
# )

In [8]:
# Class_ID = 'Dairy'
# Country = 'France'

# @pn.depends(time_range=date_range_slider_old.param.value)
# def plot_map_interactive(time_range):
#     return plot_map(ferm_locations_final, Class_ID, Country, time_range)

In [9]:
# # Set dashboard layout
# widgets = pn.Column(pn.Spacer(height=30), date_range_slider_old, width=300)

# pn.Row(plot_map_interactive, widgets)

# Bokeh (Faster)

In [10]:
# date_range_slider_demo = pn.widgets.DateRangeSlider(
#     name='Show Samples Between:',
#     bar_color = '#37F0EC',
#     start=dt.datetime(2021, 12, 1), end=dt.datetime(2022, 8, 1),
#     value=(dt.datetime(2021, 12, 1), dt.datetime(2022, 8, 1))
# )

In [11]:
# Class_ID = 'Dairy'

In [12]:
date_range_slider = pn.widgets.DatetimeRangePicker(
    name='Show Samples Between:',
    enable_time = False,
    start=dt.datetime(2021, 12, 1), end=dt.datetime(2022, 8, 1),
    value=(dt.datetime(2021, 12, 1), dt.datetime(2022, 8, 1))
)

In [13]:
country_choice = pn.widgets.MultiChoice(name='Country', value=['France', 'Thailand'],
    options=list(ferm_locations_final.Country_obtained.unique()))

In [14]:
class_choice = pn.widgets.MultiChoice(name='Class', value=['Dairy', 'Fish'],
    options=list(ferm_locations_final.Class.unique()))

In [15]:
def plot_map_bokeh(ferm_locations_final, ferm_class_list=class_choice.value, date_range=date_range_slider.value, country_list=country_choice.value):
    """Make a map in bokeh."""

    sub_df = extract_sub_df(ferm_locations_final, ferm_class_list, country_list, date_range)

    p = bokeh.plotting.figure(
        frame_height=400,
        frame_width=700,
        x_axis_label="Latitude",
        y_axis_label="Longitude",
        tools =['wheel_zoom', 'pan', 'reset', 'tap'],
        x_axis_type = 'mercator',
        y_axis_type = 'mercator',
        x_range = [ferm_locations_final.x.min()-100000, ferm_locations_final.x.max() + 100000],
        y_range = [ferm_locations_final.y.min()-100000, ferm_locations_final.y.max() + 100000]

    )

    # Set up data source; this is what gets changed in the callback
    source = bokeh.models.ColumnDataSource(
        dict(
            x=sub_df["x"].values,
            y=sub_df["y"].values,
            date=sub_df["Acquisition_date"].values,
            Class=sub_df['Class'].values,
            Country=sub_df['Country_obtained'].values,
        )
    )
    
    Class_list=ferm_locations_final.Class.unique()
    Class_len = len(Class_list)

    # Mapping of color for glyphs
    mapper = bokeh.transform.factor_cmap('Class', palette=bokeh.palettes.Category20[15], factors=Class_list)

    p.circle(source=source, x="x", y="y", color=mapper, size=7, line_alpha=0)

    p.toolbar_location = 'above'
    p.add_tile(tile_provider)
    p.add_tools(bokeh.models.BoxZoomTool(match_aspect=True))
    
    hover = bokeh.models.HoverTool()
    
    hover.tooltips = """
    <font face="Arial" size="3">
    <div>
        <div><strong>Ferment Class:  </strong>@Class</div>
        <div><strong>Name: </strong>@x</div>
    </div>
    </font>
    """
    p.add_tools(hover)
    
    
    
    url = "https://github.com/lianamerk/hovercal/blob/main/examples/fruit_hovercal_1.png?raw=true"
    taptool = p.select(type=bokeh.models.TapTool)
    taptool.callback = bokeh.models.OpenURL(url=url)

    return p


# # Take a look
# p = plot_map_bokeh(ferm_locations_final, "Dairy", "France", date_range=inital_date_range)

# bokeh.io.show(p)

In [16]:
p_pane = pn.pane.Bokeh(plot_map_bokeh(ferm_locations_final))

In [17]:
ferm_class = 'Dairy'
# country = 'France'

In [18]:
def date_interval_callback(target, event):
    ferm_class_list = class_choice.value
    country_list = country_choice.value
    date_range = event.new
    
    inds = (
        (ferm_locations_final["Class"].isin(ferm_class_list))
        & (ferm_locations_final["Country_obtained"].isin(country_list))
        & (ferm_locations_final["Acquisition_date"] >= date_range[0])
        & (ferm_locations_final["Acquisition_date"] <= date_range[1])
    )

    sub_df = ferm_locations_final.loc[inds, ["x", "y", "Acquisition_date", "Class", "Country_obtained"]]

    print(len(sub_df))
    gr = target.object.renderers[0]
    source = gr.data_source
    source.data = {name : [] for name in ['x', 'y', 'date', 'Class', 'Country']}

    source.data["x"] = sub_df["x"].values
    source.data["y"] = sub_df["y"].values
    source.data["date"] = sub_df["Acquisition_date"].values
    source.data["Class"] = sub_df["Class"].values
    source.data["Country"] = sub_df["Country_obtained"].values

    
    # print(source.data)

In [19]:
def country_callback(target, event):
    ferm_class_list = class_choice.value
    country_list = event.new
    date_range = date_range_slider.value
    
    inds = (
        (ferm_locations_final["Class"].isin(ferm_class_list))
        & (ferm_locations_final["Country_obtained"].isin(country_list))
        & (ferm_locations_final["Acquisition_date"] >= date_range[0])
        & (ferm_locations_final["Acquisition_date"] <= date_range[1])
    )

    sub_df = ferm_locations_final.loc[inds, ["x", "y", "Acquisition_date", "Class", "Country_obtained"]]

    print(sub_df)
    gr = target.object.renderers[0]
    source = gr.data_source
    source.data = {name : [] for name in ['x', 'y', 'date', 'Class', 'Country']}

    source.data["x"] = sub_df["x"].values
    source.data["y"] = sub_df["y"].values
    source.data["date"] = sub_df["Acquisition_date"].values
    source.data["Class"] = sub_df["Class"].values
    source.data["Country"] = sub_df["Country_obtained"].values
    
    # print(source.data)

In [20]:
def class_callback(target, event):
    print(event.new)
    ferm_class_list = event.new
    country_list = country_choice.value
    date_range = date_range_slider.value
    
    inds = (
        (ferm_locations_final["Class"].isin(ferm_class_list))
        & (ferm_locations_final["Country_obtained"].isin(country_list))
        & (ferm_locations_final["Acquisition_date"] >= date_range[0])
        & (ferm_locations_final["Acquisition_date"] <= date_range[1])
    )

    sub_df = ferm_locations_final.loc[inds, ["x", "y", "Acquisition_date", "Class", "Country_obtained"]]

    gr = target.object.renderers[0]
    source = gr.data_source
    source.data = {name : [] for name in source.data}

    source.data["x"] = sub_df["x"].values
    source.data["y"] = sub_df["y"].values
    source.data["date"] = sub_df["Acquisition_date"].values
    source.data["Class"] = sub_df["Class"].values
    source.data["Country"] = sub_df["Country_obtained"].values
    print(source.data)
   

In [21]:
date_range_slider.link(p_pane, callbacks={'value': date_interval_callback})

Watcher(inst=DatetimeRangePicker(enable_time=False, end=datetime.datetime(2022, ..., name='Show Samples Between:', start=datetime.datetime(2021, ..., value=(datetime.datetime(2021, ...), cls=<class 'panel.widgets.input.DatetimeRangePicker'>, fn=<function Reactive.link.<locals>.link_cb at 0x7f8ca8929c10>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [22]:
country_choice.link(p_pane, callbacks={'value': country_callback})

Watcher(inst=MultiChoice(name='Country', options=['France', 'Singapore', ...], value=['France', 'Thailand']), cls=<class 'panel.widgets.select.MultiChoice'>, fn=<function Reactive.link.<locals>.link_cb at 0x7f8ca8929b80>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [23]:
class_choice.link(p_pane, callbacks={'value': class_callback})

Watcher(inst=MultiChoice(name='Class', options=['Dairy', 'Control', ...], value=['Dairy', 'Fish']), cls=<class 'panel.widgets.select.MultiChoice'>, fn=<function Reactive.link.<locals>.link_cb at 0x7f8ca8c69160>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [24]:
map_panel = pn.Row(p_pane, pn.Spacer(width=15), pn.Column(date_range_slider, country_choice, class_choice))
map_panel

Row
    [0] Bokeh(Figure)
    [1] Spacer(width=15)
    [2] Column
        [0] DatetimeRangePicker(enable_time=False, end=datetime.datetime(2022, ..., name='Show Samples Between:', start=datetime.datetime(2021, ..., value=(datetime.datetime(2021, ...)
        [1] MultiChoice(name='Country', options=['France', 'Singapore', ...], value=['France', 'Thailand'])
        [2] MultiChoice(name='Class', options=['Dairy', 'Control', ...], value=['Dairy', 'Fish'])

In [25]:
map_panel.save('test.html', embed=True)
